# Import packages

In [1]:
import json
import pickle
import numpy as np

# Load Dataset

In [2]:
data_reader = open("../datasets/development.json", mode="r", encoding="utf-8")

# Go to beginning
data_reader.seek(0)

# Parse all text from json
documents = [document['description']
             for document in json.loads(data_reader.readline())
             if document['description']]

data_reader.close()

# Load LSH results

Both structures are organized as:

doc id - group id

Where,for both:
* doc id represent the same document
* group id do not have same meaning

## Groups to pairs converter

In [3]:
def groups_to_pairs(groups):
    pairs = set()
    for group_id, doc_ids in groups.items():
        doc_ids = list(doc_ids)
        for i in range(0, len(doc_ids)):
            for j in range(i + 1, len(doc_ids)):           
                # Keep same order, so we can compare
                if doc_ids[i] > doc_ids[j]:
                    pairs.add((doc_ids[j], doc_ids[i]))
                else:
                    pairs.add((doc_ids[i], doc_ids[j]))
    # Order so it's easier to compare
    # pairs = sorted(list(pairs), key=lambda element: (element[0], element[1]))
    return pairs

## Load Matheus results

In [4]:
with open("grupos_Matheus.pickle", "rb") as handle:
    matheus_by_group = pickle.load(handle)
    matheus_by_doc = {doc_id:group_id                      
                      for group_id, doc_ids in matheus_by_group.items()
                      for doc_id in doc_ids}
    matheus_by_pairs = groups_to_pairs(matheus_by_group)

## Load Thiago results

In [5]:
with open("grupos_Thiago.pickle", "rb") as handle:
    thiago_by_doc = pickle.load(handle)
    thiago_by_group = {}
    for doc_id, group_id in thiago_by_doc.items():
        if group_id not in thiago_by_group:
            thiago_by_group[group_id] = set([doc_id])
        else:
            thiago_by_group[group_id].add(doc_id)
    thiago_by_pairs = groups_to_pairs(thiago_by_group)

# Comparison Study

## Pairs

### Compute

In [6]:
union_pairs = matheus_by_pairs | thiago_by_pairs
intersection_pairs = matheus_by_pairs & thiago_by_pairs
matheus_extra_pairs = matheus_by_pairs - thiago_by_pairs
thiago_extra_pairs = thiago_by_pairs - matheus_by_pairs

### Statistics

In [7]:
print("Pairs found by (Quantitative):")
print("Both\t %d" % len(intersection_pairs))
print("Matheus\t %d" % len(matheus_extra_pairs))
print("Thiago\t %d" % len(thiago_extra_pairs))
print("Total\t %d" % len(union_pairs))
print("")
print("Pairs found by (Percentage):")
print("Both\t %.2f%%" % (100 * len(intersection_pairs)/len(union_pairs),))
print("Matheus\t %.2f%%" % (100 * len(matheus_extra_pairs)/len(union_pairs),))
print("Thiago\t %.2f%%" % (100 * len(thiago_extra_pairs)/len(union_pairs),))

Pairs found by (Quantitative):
Both	 3016
Matheus	 351
Thiago	 549
Total	 3916

Pairs found by (Percentage):
Both	 77.02%
Matheus	 8.96%
Thiago	 14.02%


### Examples of extra pairs 

In [8]:
def show_random(pairs, size):
    for _ in range(size):
        idx = np.random.randint(0, len(pairs))
        item_a, item_b = pairs[idx]
        print('%s:\t%s' % (item_a, documents[item_a]))
        print('%s:\t%s' % (item_b, documents[item_b]))
        print()

#### By Matheus

In [25]:
show_random(list(matheus_extra_pairs), 1)

1062:	Empresa contrata profissional com superior completo em engenharia (preferencialmente engenharia eletrica). Atribuicoes: elaborar e implementar acoes preventivas e corretivas; controlar e supervisionar trabalho das empresas terceirizadas de manutencao; elaborar relatorios de controle gerencial; acompanhar o consumo de materiais, utilizacao de equipamentos e outros insumos; entre outras atividades da funcao. Salario a combinar, acrescidos de beneficios ( plano de saúde, odontologico, ticket refeicao e ppr).
2737:	Empresa contrata profissional com superior completo em engenharia (preferencialmente engenharia eletrica). Atribuicoes: elaborar e implementar acoes preventivas e corretivas; controlar e supervisionar  trabalho das empresas terceirizadas de manutencao; elaborar relatorios de controle gerencial; acompanhar o consumo de materiais, utilizacao de equipamentos e outros insumos; entre outras atividades da funcao. Salario a combinar, acrescidos de beneficios ( plano de saude, odo

In [22]:
show_random(list(thiago_extra_pairs), 1)

4528:	:confidencialagv logistica esta com 1 vaga(s) em aberto para vinhedo / sp.responsabilidades: – conhecimento em informatica (excel, libreoffice) - bom relacionamento interpessoal para trabalho em equipe - habilidade para falar em publico, pois ira ministrar treinamentos.requisitos: – experiencia com abertura e controle de nao conformidades - qualificacao de fornecedores - conhecimento em boas praticas - auditoria de clientessalario: a combinarbeneficios: vale transporte/fretado, refeicao no local, vale alimentacao, convenio medico e odontologico.observacoes: enviar último salario e pretensao salarial.
6267:	nome fantasia:confidencial
agv logistica esta com 1 vaga(s) em aberto para vinhedo / sp.responsabilidades: – conhecimento em informatica (excel, libreoffice) - bom relacionamento interpessoal para trabalho em equipe - habilidade para falar em publico, pois ira ministrar treinamentos.requisitos: – experiencia com abertura e controle de nao conformidades - qualificacao de fornece